# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

# Result Filename

In [2]:
result_filename='AWS_Results_CelebDPR.csv'

# Imports

In [3]:
import cv2
import os
import matplotlib.pyplot as plt
import glob
import time
import gc
import pandas as pd
import numpy as np
import boto3
from PIL import Image
import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import ImageDraw, ImageFont
from glob import glob
from pprint import pprint
import pprint
from difflib import SequenceMatcher

import csv
from time import sleep
from glob import glob
from pprint import pprint

# Initialization

In [4]:
BUCKET = "amazon-rekognition"
KEY = "search.jpg"
COLLECTION = "my-collection-id"

ACCESS_ID = ''
ACCESS_KEY = ''

client=boto3.client('rekognition', region_name='ap-northeast-2', aws_access_key_id=ACCESS_ID, aws_secret_access_key=ACCESS_KEY)
# rekognition = boto3.client('rekognition')

# img_subtraction_folder = '/media/data2/dataset/DF/Fake_Localized'

# if not os.path.exists(img_subtraction_folder):
#     os.mkdir(img_subtraction_folder)

# Methods

In [5]:
def compare_faces(sourceFile, targetFile):
    imageSource = open(sourceFile, 'rb')
    imageTarget = open(targetFile, 'rb')
    try:
        response = client.compare_faces(SimilarityThreshold=0,
                                    SourceImage={'Bytes': imageSource.read()},
                                    TargetImage={'Bytes': imageTarget.read()})
        for faceMatch in response['FaceMatches']:
            similarity = str(faceMatch['Similarity'])
            return similarity
        return 0
    except:
        error = "can't detect"
        return 0
    f.close()
    imageSource.close()
    imageTarget.close()
def recognize_celebrities(photo):
    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})
        for celebrity in response['CelebrityFaces']:
            return celebrity['Name'],celebrity['MatchConfidence']
        return "",""
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].split("_")[0]
def get_name_tar(filename):
    return os.path.basename(os.path.dirname(filename))

In [ ]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity", ])
file.close()

In [9]:
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/fake/*")
count=0
skip=0
for actor in actors_list:
    count+=1
    if skip ==0:
        if os.path.basename(actor) =="Sofía Vergara":
            skip=1
        else:
            continue
#     print(os.path.basename(actor))
#     break
    real_image=os.path.join("/home/sha/project/adversarial_deepfake/src/Similarity/CelebDPR/real",
                                os.path.basename(actor)+".jpg")
    fake_images_list=glob(os.path.join(actor,"*"))
    print(real_image)    
    result_real_pred,result_real_conf=recognize_celebrities(real_image)
    src=get_name(real_image)
    for fake_image in fake_images_list:
        tar=get_name_tar(fake_image)
        result_fake_pred,result_fake_conf=recognize_celebrities(fake_image)
        similarity=compare_faces(real_image,fake_image)
        print(src,os.path.basename(fake_image),result_fake_pred,result_fake_conf,similarity)
        file=open(result_filename, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow([fake_image,src,tar,
              result_real_pred,result_real_conf,
              result_fake_pred,result_fake_conf,
              similarity])
        file.close()
print("Total:",count)